In [ ]:
import os
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
from pathlib import Path

#weather
weather_df = pd.read_csv("../data/raw/meteostat/dia_2023-2024_hourly.csv")

#departures
bts_departures = Path("../data/raw/bts/departure_stats")
departure_files = list(bts_departures.glob("*csv"))
departures_df = pd.concat((pd.read_csv(f) for f in departure_files))

#bts t100

t100_column_key = ["Year","Month", "OA_Alpha_Code", "OA_Unique_Numeric_Code",
                   "OA World Area Code", "OA City Name", "DA Alpha Code","DA Airport Numeric Code",
                   "DA World Area Code", "DA City Name", "Carrier Alpha Code", "Carrier Entity Code", 
                   "OAI Group Code", "Distance","Service Class", "Aircraft Group","Aircraft Type",
                   "Aircraft Configuration","Departures Performed", "Departures Scheduled", "Available Capacity",
                   "Available Seats", "Passenger Transported","Freight Transported","Mail Transported","Ramp-Ramp Minutes",
                   "Airborne Minutes", "Carrier World Area Code"]
bts_t100 = Path("../data/raw/bts/t100")
t100_files = list(bts_t100.glob("*.asc"))
t100_df = pd.concat((pd.read_table(f, sep= "|", header = None) for f in t100_files))
t100_df = t100_df.drop(columns=[28])
t100_df.columns = t100_column_key
#filter down to just DIA rows (DEN but ill never consider it such)
destination = t100_df.loc[t100_df['DA City Name'] == "Denver, CO"]
arrival = t100_df.loc[t100_df['OA City Name'] == "Denver, CO"]
t100_df = pd.concat([destination, arrival],ignore_index=True)

# print(weather_df.head(2))
# print("departures")
# print(departures_df.head(2))
# print("t100")
# print(t100_df.head(2))

<class 'pandas.core.frame.DataFrame'>
Index: 26790 entries, 838 to 436770
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Year                     26790 non-null  int64 
 1   Month                    26790 non-null  int64 
 2   OA_Alpha_Code            26790 non-null  object
 3   OA_Unique_Numeric_Code   26790 non-null  int64 
 4   OA World Area Code       26790 non-null  int64 
 5   OA City Name             26790 non-null  object
 6   DA Alpha Code            26790 non-null  object
 7   DA Airport Numeric Code  26790 non-null  int64 
 8   DA World Area Code       26790 non-null  int64 
 9   DA City Name             26790 non-null  object
 10  Carrier Alpha Code       26790 non-null  object
 11  Carrier Entity Code      26790 non-null  object
 12  OAI Group Code           26790 non-null  int64 
 13  Distance                 26790 non-null  int64 
 14  Service Class            26790 non-null 

In [5]:
#make date column into datetime and sort arrival/depart chronologically
weather_df["time"] = pd.to_datetime
departures_df["Date (MM/DD/YYYY)"] = pd.to_datetime(departures_df["Date (MM/DD/YYYY)"])